In [2]:
from pyspark.sql import SparkSession

file_path = 'Files/ipldata/ipl_summary_raw.csv'

df = spark.read.csv(file_path, header=True, inferSchema=True)

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 4, Finished, Available, Finished)

In [3]:
df_partitioned = df.repartition("info_season")

df_partitioned.write.parquet("Files/data/ipl_partitioned")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 5, Finished, Available, Finished)

In [4]:
df_partitioned = df.repartition("info_season")

df_partitioned.write.saveAsTable("ipl_partitioned_table")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 6, Finished, Available, Finished)

In [5]:
df_partitioned = df.repartition("info_season")

df_partitioned.write.format("delta").saveAsTable("ipl_partitioned_delta_table")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql import Row

city_dimension_data = [
    Row(info_city="Hyderabad", city_population=10000000, city_state="Telangana", city_country="India"),
    Row(info_city="Bengaluru", city_population=12000000, city_state="Karnataka", city_country="India"),
    Row(info_city="Chennai", city_population=7000000, city_state="Tamil Nadu", city_country="India"),
]

city_dimension_df = spark.createDataFrame(city_dimension_data)
city_dimension_df.show()

from pyspark.sql.functions import when

left_joined_df = df.join(city_dimension_df, on="info_city", how="left")

df_replaced = df.withColumn("info_city", when(df.info_city == "Bangalore", "Bengaluru").otherwise(df.info_city))

left_joined_df = df_replaced.join(city_dimension_df, on="info_city", how="left").select("info_city", "info_outcome_winner", "city_state")

left_joined_df.show(truncate=False)

city_dimension_df.write.saveAsTable("city_dimension")
left_joined_df.write.saveAsTable ("winner_by_state")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 8, Finished, Available, Finished)

+---------+---------------+----------+------------+
|info_city|city_population|city_state|city_country|
+---------+---------------+----------+------------+
|Hyderabad|       10000000| Telangana|       India|
|Bengaluru|       12000000| Karnataka|       India|
|  Chennai|        7000000|Tamil Nadu|       India|
+---------+---------------+----------+------------+

+---------+---------------------------+----------+
|info_city|info_outcome_winner        |city_state|
+---------+---------------------------+----------+
|Mumbai   |Mumbai Indians             |null      |
|Mumbai   |Mumbai Indians             |null      |
|Mumbai   |Mumbai Indians             |null      |
|Kolkata  |Kolkata Knight Riders      |null      |
|Kolkata  |Kolkata Knight Riders      |null      |
|Pune     |Rising Pune Supergiant     |null      |
|Pune     |Delhi Daredevils           |null      |
|Delhi    |Delhi Daredevils           |null      |
|Delhi    |Kolkata Knight Riders      |null      |
|Bengaluru|Royal Challe

In [7]:
spark.sql("SELECT * FROM city_dimension").show()

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 9, Finished, Available, Finished)

+---------+---------------+----------+------------+
|info_city|city_population|city_state|city_country|
+---------+---------------+----------+------------+
|Hyderabad|       10000000| Telangana|       India|
|Bengaluru|       12000000| Karnataka|       India|
|  Chennai|        7000000|Tamil Nadu|       India|
+---------+---------------+----------+------------+



In [8]:
spark.sql("SELECT * FROM winner_by_state WHERE info_city ='Chennai'").show()

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 10, Finished, Available, Finished)

+---------+--------------------+----------+
|info_city| info_outcome_winner|city_state|
+---------+--------------------+----------+
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai|      Mumbai Indians|Tamil Nadu|
|  Chennai| Chennai Super Kings|Tamil Nadu|
|  Chennai|      Mumbai Indians|Tamil Nadu|
|  Chennai|Royal Challengers...|Tamil Nadu|
|  Chennai|Kolkata Knight Ri...|Tamil Nadu|
|  Chennai|      Mumbai Indians|Tamil Nadu|
|  Chennai|Royal Challengers...|Tamil Nadu|
|  Chennai|      Mumbai Indians|Tamil Nadu|
|  Chennai|Royal Challengers...|Tamil Nadu|
|  Chennai|      Delhi Capitals|Tamil Nadu|
|  Chennai| Sunrisers Hyderabad|Tamil Nadu|
|  Chennai|        Punjab Kings|Tamil Nadu|
|  Chennai|                null|Tamil Nadu|
|  Chennai| Chennai Super Kings|

In [9]:
from pyspark.sql import Row

city_dimension_data = [
    Row(info_city="Hyderabad", city_population=10000000, city_state="Telangana", city_country="India"),
    Row(info_city="Bengaluru", city_population=12000000, city_state="Karnataka", city_country="India"),
    Row(info_city="Chennai", city_population=7000000, city_state="Tamil Nadu", city_country="India"),
    Row(info_city="Mumbai", city_population=7000000, city_state="Maharashtra", city_country="India"),
    Row(info_city="Chandigarh", city_population=7000000, city_state="Punjab", city_country="India")
]

city_dimension_df = spark.createDataFrame(city_dimension_data)
city_dimension_df.show()

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 11, Finished, Available, Finished)

+----------+---------------+-----------+------------+
| info_city|city_population| city_state|city_country|
+----------+---------------+-----------+------------+
| Hyderabad|       10000000|  Telangana|       India|
| Bengaluru|       12000000|  Karnataka|       India|
|   Chennai|        7000000| Tamil Nadu|       India|
|    Mumbai|        7000000|Maharashtra|       India|
|Chandigarh|        7000000|     Punjab|       India|
+----------+---------------+-----------+------------+



In [10]:
city_dimension_df.write.mode("overwrite").saveAsTable("city_dimension")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 12, Finished, Available, Finished)

In [11]:
from pyspark.sql.functions import when

left_joined_df = df.join(city_dimension_df, on="info_city", how="left")

df_replaced = df.withColumn("info_city", when(df.info_city == "Bangalore", "Bengaluru").otherwise(df.info_city))

left_joined_df = df_replaced.join(city_dimension_df, on="info_city", how="left").select("info_city", "info_outcome_winner", "city_state")

left_joined_df.show(truncate=False)
left_joined_df.write.mode("overwrite").saveAsTable("winner_by_state")

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 13, Finished, Available, Finished)

+---------+---------------------------+-----------+
|info_city|info_outcome_winner        |city_state |
+---------+---------------------------+-----------+
|Mumbai   |Mumbai Indians             |Maharashtra|
|Mumbai   |Mumbai Indians             |Maharashtra|
|Mumbai   |Mumbai Indians             |Maharashtra|
|Kolkata  |Kolkata Knight Riders      |null       |
|Kolkata  |Kolkata Knight Riders      |null       |
|Pune     |Rising Pune Supergiant     |null       |
|Pune     |Delhi Daredevils           |null       |
|Delhi    |Delhi Daredevils           |null       |
|Delhi    |Kolkata Knight Riders      |null       |
|Bengaluru|Royal Challengers Bangalore|Karnataka  |
|Bengaluru|Mumbai Indians             |Karnataka  |
|Bengaluru|Rising Pune Supergiant     |Karnataka  |
|Hyderabad|Sunrisers Hyderabad        |Telangana  |
|Hyderabad|Sunrisers Hyderabad        |Telangana  |
|Hyderabad|Sunrisers Hyderabad        |Telangana  |
|Hyderabad|Sunrisers Hyderabad        |Telangana  |
|Rajkot   |K

In [12]:
spark.sql("SELECT * FROM winner_by_state WHERE info_city ='Mumbai'").show()

StatementMeta(, 830fcb1e-48b9-425c-93a3-8a2f1f97ed0c, 14, Finished, Available, Finished)

+---------+--------------------+-----------+
|info_city| info_outcome_winner| city_state|
+---------+--------------------+-----------+
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|Rising Pune Super...|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|     Kings XI Punjab|Maharashtra|
|   Mumbai|Rising Pune Super...|Maharashtra|
|   Mumbai| Chennai Super Kings|Maharashtra|
|   Mumbai|    Delhi Daredevils|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai| Sunrisers Hyderabad|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai|    Rajasthan Royals|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai| Chennai Super Kings|Maharashtra|
|   Mumbai| Chennai Super Kings|Maharashtra|
|   Mumbai|      Delhi Capitals|Maharashtra|
|   Mumbai|      Mumbai Indians|Maharashtra|
|   Mumbai